In [14]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd
import regex as re

In [15]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
position = 'python'
jobs = []

###  HH 

In [16]:
main_link_hh ='https://hh.ru/search/vacancy?area=2&st=searchVacancy&text='

In [17]:
for i in range(2):
    html_hh = requests.get(f'{main_link_hh}{position.replace(" ", "+")}&page={i}', headers=headers).text
    html_hh = bs(html_hh, 'lxml')

    vacancy_list_hh = html_hh.find_all('div', {'data-qa':['vacancy-serp__vacancy', 'vacancy-serp__vacancy vacancy-serp__vacancy_premium']})

    for vacancy in vacancy_list_hh:
        vacancy_data = {}
        title_block = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'})
        salary_block = vacancy.find('div',{'class':'vacancy-serp-item__compensation'})

        vacancy_title = title_block.getText()
        vacancy_link = title_block['href']
        site = 'HeadHunter'

        if salary_block is None:
            salary_min = 'None'
            salary_max = 'None'
        elif 'от' in salary_block.getText():
            salary_min = re.findall("\d.+", salary_block.getText())[0].replace('\xa0',' ')
            salary_max = 'None'
        elif 'до' in salary_block.getText():
            salary_max = re.findall("\d.+", salary_block.getText())[0].replace('\xa0',' ')
            salary_min = 'None'
        elif '-' in salary_block.getText():
            salary_min = re.findall("^\d+.\d+", salary_block.getText())[0].replace('\xa0',' ') + ' руб.'
            salary_max = re.findall("\d+.\d+\w+", salary_block.getText())[1].replace('\xa0',' ') + ' руб.'

        vacancy_data['title'] = vacancy_title
        vacancy_data['job_link'] = vacancy_link
        vacancy_data['salary_min'] = salary_min
        vacancy_data['salary_max'] = salary_max
        vacancy_data['source'] = site
        jobs.append(vacancy_data)

###  SUPERJOB

In [18]:
main_link_sj = 'https://www.superjob.ru'

In [19]:
for i in range(2):
    html_sj = requests.get(f'{main_link_sj}/vacancy/search/?keywords={position.replace(" ", "%20")}&geo%5Bt%5D%5B0%5D=14&page={i+1}', headers=headers).text
    html_sj = bs(html_sj, 'lxml')

    vacancy_list_sj = html_sj.findAll('div',{'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})

    for vacancy in vacancy_list_sj:
        vacancy_data = {}
        title_block = vacancy.find('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
        salary_block = vacancy.find('span',{'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})

        vacancy_title = title_block.getText()
        vacancy_link = main_link_sj + vacancy.find('a',{'target':'_blank'})['href']
        site = 'SuperJob'

        if salary_block is not None:
            if 'от' in salary_block.getText():
                salary_min = re.findall("\d.+", salary_block.getText())[0].replace('\xa0',' ')
                salary_max = 'None'
            elif ' — ' in salary_block.getText():
                salary_min = re.findall("^\d+.\d+", salary_block.getText())[0].replace('\xa0',' ') + ' руб.'
                salary_max = re.findall("\d+.\d+\w+", salary_block.getText())[1].replace('\xa0',' ') + ' руб.'
            else:
                salary_min = salary_block.getText()
                salary_max = salary_block.getText()
        else:
            salary_min = 'None'
            salary_max = 'None'

        vacancy_data['title'] = vacancy_title
        vacancy_data['job_link'] = vacancy_link
        vacancy_data['salary_min'] = salary_min
        vacancy_data['salary_max'] = salary_max
        vacancy_data['source'] = site

        jobs.append(vacancy_data)

## Выводим все значения

In [20]:
pprint(jobs)

[{'job_link': 'https://hh.ru/vacancy/33206558?query=python',
  'salary_max': 'None',
  'salary_min': 'None',
  'source': 'HeadHunter',
  'title': 'Senior Data Scientist (Команда знаний)'},
 {'job_link': 'https://hh.ru/vacancy/32370885?query=python',
  'salary_max': '150 000 руб.',
  'salary_min': '120 000 руб.',
  'source': 'HeadHunter',
  'title': 'QA Automation Engineer'},
 {'job_link': 'https://hh.ru/vacancy/34200108?query=python',
  'salary_max': 'None',
  'salary_min': '120 000 руб.',
  'source': 'HeadHunter',
  'title': 'Full-stack developer'},
 {'job_link': 'https://hh.ru/vacancy/34411865?query=python',
  'salary_max': '160 000 руб.',
  'salary_min': '80 000 руб.',
  'source': 'HeadHunter',
  'title': 'Python разработчик'},
 {'job_link': 'https://hh.ru/vacancy/34411989?query=python',
  'salary_max': '3 000 EUR',
  'salary_min': 'None',
  'source': 'HeadHunter',
  'title': 'Python разработчик'},
 {'job_link': 'https://hh.ru/vacancy/34274386?query=python',
  'salary_max': 'None',
